In [26]:
import os
import re
import sys
import csv
import time
import tqdm
import nltk
import math
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import tree
from sklearn import datasets
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.corpus import wordnet
from scipy.stats import uniform
from scipy.stats import randint


vectorizer = CountVectorizer()

corpus_train = pd.read_csv("reddit_train.csv",usecols=['comments','subreddits'],delimiter=',')
corpus_test = pd.read_csv("reddit_test.csv",usecols=['comments'],delimiter=',')
english_words = set(nltk.corpus.words.words()) # important for cleaning non-english tokens


# a helper function to process one comment
def preprocess_text(text): 
    text = text.lower().split()
    stops = set(stopwords.words("english"))
#     text = [w for w in text if not w in stops and len(w) >= 3]
    text = " ".join(text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"^https?:\/\/.*[\r\n]*","", text)
    text = text.split()
    
    # lemmatization
    lemma = nltk.wordnet.WordNetLemmatizer()
    text = [lemma.lemmatize(word) for word in text]
    
    # stemming
    text = [PorterStemmer().stem(word) for word in text]
    
    
    text_final = []
    
    # clean all non-English words, numbers, and other weirdos, stopwords
    for x in text:
        if x.isalpha() and x not in stops and x in english_words:
            text_final.append(x)
    
    text = " ".join(text_final)
    return text


# the major function to process the training dataset
# returns (1) a matrix of all training features x (2) a numpy array of y labels
def preprocess():
    df = corpus_train.copy()
    df['comments'] = df['comments'].map(lambda x: preprocess_text(x))
    y_train = df["subreddits"].to_numpy()
    x_train = vectorizer.fit_transform(df['comments'])
    #print(vectorizer.get_feature_names())
    return x_train, y_train

# function to process the testing set
# returns the matrix of all testing features
def preprocess_testing():
    df = corpus_test.copy()
    df['comments'] = df['comments'].map(lambda x: preprocess_text(x))
    #y_train = df["subreddits"].to_numpy()
    x_train = vectorizer.fit_transform(df['comments'])
    return x_train
    

In [27]:
x,y = preprocess()
print(x.shape)
y2 = preprocess_testing()
print(y2.shape)

(70000, 10323)
(30000, 8236)


In [175]:
print(vectorizer.get_feature_names())

  (0, 22632)	1
  (0, 6394)	3
  (0, 25040)	1
  (0, 49543)	5
  (0, 10205)	1
  (0, 1908)	1
  (0, 40958)	1
  (0, 36519)	1
  (0, 25734)	1
  (0, 49523)	1
  (0, 29720)	1
  (0, 18126)	3
  (0, 47055)	1
  (0, 20038)	1
  (0, 53925)	3
  (0, 19121)	1
  (0, 19628)	1
  (0, 49755)	1
  (0, 14772)	1
  (0, 18629)	1
  (0, 34423)	1
  (0, 50873)	1
  (0, 4620)	1
  (0, 36054)	1
  (0, 34386)	2
  (0, 55815)	1
  (0, 6632)	1
  (0, 48625)	1
  (0, 12488)	1
  (0, 49354)	1
  (0, 9358)	1
  (0, 32228)	1
  (0, 31961)	1
  (0, 41375)	1
  (0, 7811)	1
  (0, 50212)	1
  (0, 13961)	1
  (0, 28870)	1
{'movies', 'baseball', 'nba', 'canada', 'hockey', 'Overwatch', 'GlobalOffensive', 'wow', 'gameofthrones', 'worldnews', 'Music', 'funny', 'nfl', 'europe', 'soccer', 'trees', 'AskReddit', 'anime', 'conspiracy', 'leagueoflegends'}


In [152]:
print(vectorizer.get_feature_names())

['aa', 'aaa', 'aaaa', 'aaaaa', 'aaaaaa', 'aaaaaaaaaaaaand', 'aaaaaaaaahq', 'aaaaaaaabeq', 'aaaaaaaacxi', 'aaaaaaaaezo', 'aaaaaaaafii', 'aaaaaaaamza', 'aaaaaaaannd', 'aaaaaaaaz', 'aaaaaaabheo', 'aaaaaand', 'aaaaand', 'aaaages', 'aaaah', 'aaaand', 'aaaannnd', 'aaah', 'aaahahaha', 'aaand', 'aaannnd', 'aaawhitney', 'aabb', 'aac', 'aacnjournals', 'aaf', 'aafc', 'aah', 'aahhh', 'aahhhh', 'aaliyah', 'aam', 'aan', 'aang', 'aangechouwen', 'aardman', 'aaron', 'aarp', 'aarya', 'aasaults', 'aat', 'aatrox', 'aau', 'aav', 'aawe', 'ab', 'aba', 'aback', 'abadi', 'abam', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandonned', 'abandonware', 'abaya', 'abb', 'abbas', 'abbey', 'abbreviated', 'abbreviation', 'abby', 'abc', 'abck', 'abd', 'abdelhak', 'abdelkader', 'abdicate', 'abdomen', 'abdominal', 'abducted', 'abducting', 'abduction', 'abductive', 'abdul', 'abdullah', 'abe', 'abeast', 'abed', 'abel', 'abell', 'abendblatt', 'aberation', 'aberdeen', 'aberg', 'aberration', 'abetting', 'abeyta', 'a

In [11]:
vectorizer = CountVectorizer()

corpus = [ 'This is a sentence',
           'Another sentence is here',
           'Wait for another sentence',
           'The sentence is coming',
           'The sentence has come'
         ]

x = vectorizer.fit_transform(corpus)
print(pd.DataFrame(x.A, columns=vectorizer.get_feature_names()).to_string())

   another  come  coming  for  has  here  is  sentence  the  this  wait
0        0     0       0    0    0     0   1         1    0     1     0
1        1     0       0    0    0     1   1         1    0     0     0
2        1     0       0    1    0     0   0         1    0     0     1
3        0     0       1    0    0     0   1         1    1     0     0
4        0     1       0    0    1     0   0         1    1     0     0


In [13]:
print(x.A)

[[0 0 0 0 0 0 1 1 0 1 0]
 [1 0 0 0 0 1 1 1 0 0 0]
 [1 0 0 1 0 0 0 1 0 0 1]
 [0 0 1 0 0 0 1 1 1 0 0]
 [0 1 0 0 1 0 0 1 1 0 0]]


In [ ]:
#######################Stats memo##########################
No. of tokens: training set / test set

No operation: 62853 / 41462
    
remove stopwords: 62721 / 41331
remove non-English: 20713 / 16111
remove non-Englsih/remove stopwords: 20587 / 15986
    
lemma/remove non-English: 21125 / 16518
stem/remove non-English: 10462 / 8352

    
lemma/remove non-English/remove stopwords: 21002 / 16396
stem/remove non-English/remove stopwords: 10359 / 8250

lemma/stem/remove non-English/remove stopwords: 10323 / 8236


    
